In [1]:
import sys
sys.path.append('../')
sys.path.append('../../60_gebastel/process_masks/')
from helper import load_dotenv
from mask_functions import load_masks
import sqlite3
import pandas as pd
import numpy as np
from file_interaction import get_related_filepath, open_img
from tqdm.auto import tqdm

In [2]:
dotenv = load_dotenv()
con = sqlite3.connect( dotenv['DB_PATH'] )

In [3]:
def update_k_coverage( mask_id, k_mean_coverage, k_std_coverage ):
    c = con.cursor()
    c.execute(
        f'''
            UPDATE mask
            SET K_mean_coverage={ k_mean_coverage }, K_std_coverage={ k_std_coverage }
            WHERE mask_id='{ mask_id }'
        '''
    )
    c.close()
    con.commit()

In [4]:
def process_page( page ):
    img_path = get_related_filepath(
        page.job,
        'halftone600dpi',
        f'{ page.pdf_filename }.4c.jpg'
    )
    masks_path = get_related_filepath(
        page.job,
        'halftone600dpi',
        f'{ page.pdf_filename }.masks.pkl'
    )
    
    pkl_masks = load_masks(masks_path)
    img = open_img(img_path)

    relevant_masks = masks.loc[
        (masks.pdf_filename == page.pdf_filename) &
        (masks.job == page.job)
    ].index

    for rm_idx in relevant_masks:
        relevant_pkl_masks = [
            m for m in pkl_masks
            if ";".join([str(val) for val in m['bbox']]) == masks.loc[rm_idx].bbox
        ]

        if len(relevant_pkl_masks) > 0:
            pkl_mask = relevant_pkl_masks[0]
        
            cropped_img = np.array(img.crop((
                pkl_mask['bbox'][0], pkl_mask['bbox'][1],
                pkl_mask['bbox'][0]+pkl_mask['bbox'][2], pkl_mask['bbox'][1]+pkl_mask['bbox'][3]
            )))[:,:,3]
            k_mean_coverage = cropped_img[pkl_mask['mask']].mean() / 255
            k_std_coverage = cropped_img[pkl_mask['mask']].std() / 255

            update_k_coverage(
                masks.loc[rm_idx].mask_id,
                k_mean_coverage,
                k_std_coverage
            )

In [5]:
masks = pd.read_sql(
    '''
        SELECT * FROM mask
        WHERE K_mean_coverage IS NULL OR K_std_coverage IS NULL
    ''',
    con
)

pages = masks.loc[
    :,
    ['pdf_filename','job']
].drop_duplicates()

In [6]:
for i in tqdm( range(pages.shape[0]) ):
    try:
        process_page( pages.iloc[i] )
    except:
        print( pages.iloc[i].job, pages.iloc[i].pdf_filename )

  0%|          | 0/997 [00:00<?, ?it/s]

/Users/frederic.birwe/Library/Python/3.10/lib/python/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (127715826 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/Users/frederic.birwe/Library/Python/3.10/lib/python/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (91411680 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/Users/frederic.birwe/Library/Python/3.10/lib/python/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (124399302 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/Users/frederic.birwe/Library/Python/3.10/lib/python/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (120330965 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/Users/frederic.birwe/Library/Python/3.10/lib/python/site-packages/PI

24-03-05-01_randomTrainPages 617104.tg39_bg96_p1000_D1_149SA.DE.932.UM.16.gvkfpr50_PRINT 03.11..p1


/Users/frederic.birwe/Library/Python/3.10/lib/python/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (107205356 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
